# Polymarket Copy Trading Bot - Analysis

This notebook provides detailed analysis of the copy trading bot performance.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
sys.path.append('..')

from src.config import *
from src.market import MarketSimulator
from src.leader import LeaderTrader, MarketEvaluator
from src.simulator import TradingSimulator
from src.metrics import MetricsCalculator

# Set style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Generate Market Data and Leader Trades

In [ ]:
# Initialize components
market_sim = MarketSimulator(initial_price=40000.0, volatility=0.005, seed=42)
leader = LeaderTrader(skill_level=0.65, position_size=100.0, seed=42)

# Generate data
start_time = datetime.now() - timedelta(minutes=15 * 1000)
market_data = market_sim.generate_market_data(1000, start_time)
leader_trades = leader.generate_trades(market_data)

print(f"Generated {len(market_data)} market data points")
print(f"Generated {len(leader_trades)} leader trades")

# Display samples
display(market_data.head())
display(leader_trades.head())

## 2. Visualize BTC Price Movement

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Price chart
axes[0].plot(market_data['timestamp'], market_data['btc_price'], linewidth=1.5)
axes[0].set_title('BTC Price Over Time (15-minute resolution)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('BTC Price (USD)')
axes[0].grid(True, alpha=0.3)

# Outcome distribution
outcome_counts = market_data['actual_outcome'].value_counts()
axes[1].bar(outcome_counts.index, outcome_counts.values, color=['green', 'red'], alpha=0.7)
axes[1].set_title('Market Outcome Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Outcome')
axes[1].set_ylabel('Count')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"UP outcomes: {outcome_counts.get('UP', 0)} ({outcome_counts.get('UP', 0)/len(market_data)*100:.1f}%)")
print(f"DOWN outcomes: {outcome_counts.get('DOWN', 0)} ({outcome_counts.get('DOWN', 0)/len(market_data)*100:.1f}%)")

## 3. Run Backtest Simulation

In [ ]:
# Initialize simulator
simulator = TradingSimulator(
    starting_balance=10000.0,
    max_position_size=0.1,
    trading_fee=0.02,
    confidence_threshold=0.5
)

# Run backtest
trades_df = simulator.run_backtest(market_data, leader_trades)

print(f"\nExecuted {len(trades_df)} trades")
print(f"Final balance: ${simulator.get_final_balance():.2f}")
print(f"Total return: {simulator.get_total_return():.2f}%")

# Display sample trades
display(trades_df.head(10))

## 4. Calculate Performance Metrics

In [ ]:
metrics_calc = MetricsCalculator()
metrics = metrics_calc.calculate_all_metrics(trades_df, 10000.0)

# Print metrics
metrics_calc.print_metrics(metrics)

## 5. Visualize Trading Performance

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Balance over time
axes[0, 0].plot(trades_df['timestamp'], trades_df['balance'], linewidth=2, color='blue')
axes[0, 0].axhline(y=10000, color='red', linestyle='--', alpha=0.5, label='Starting Balance')
axes[0, 0].set_title('Account Balance Over Time', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Time')
axes[0, 0].set_ylabel('Balance (USD)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Cumulative PnL
trades_df['cumulative_pnl'] = trades_df['pnl'].cumsum()
axes[0, 1].plot(trades_df['timestamp'], trades_df['cumulative_pnl'], linewidth=2, color='green')
axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[0, 1].set_title('Cumulative PnL Over Time', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Cumulative PnL (USD)')
axes[0, 1].grid(True, alpha=0.3)

# PnL distribution
axes[1, 0].hist(trades_df['pnl'], bins=50, color='purple', alpha=0.7, edgecolor='black')
axes[1, 0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_title('PnL Distribution', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('PnL (USD)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Win/Loss distribution
win_loss = trades_df['correct'].value_counts()
colors = ['green', 'red']
labels = ['Wins', 'Losses']
axes[1, 1].pie(win_loss.values, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
axes[1, 1].set_title('Win/Loss Distribution', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

## 6. Confusion Matrix Visualization

In [ ]:
from sklearn.metrics import confusion_matrix

# Create confusion matrix
cm = confusion_matrix(trades_df['actual'], trades_df['predicted'], labels=['UP', 'DOWN'])

# Plot
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['UP', 'DOWN'], yticklabels=['UP', 'DOWN'],
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix: UP/DOWN Predictions', fontsize=14, fontweight='bold')
plt.ylabel('Actual Outcome')
plt.xlabel('Predicted Outcome')
plt.tight_layout()
plt.show()

# Print detailed metrics
print(f"\nTrue Positives (UP predicted, UP actual): {metrics['confusion_matrix']['TP']}")
print(f"True Negatives (DOWN predicted, DOWN actual): {metrics['confusion_matrix']['TN']}")
print(f"False Positives (UP predicted, DOWN actual): {metrics['confusion_matrix']['FP']}")
print(f"False Negatives (DOWN predicted, UP actual): {metrics['confusion_matrix']['FN']}")
print(f"\nAccuracy: {metrics['accuracy']:.2f}%")

## 7. Drawdown Analysis

In [ ]:
# Calculate running max and drawdown
trades_df['running_max'] = trades_df['cumulative_pnl'].cummax()
trades_df['drawdown'] = trades_df['running_max'] - trades_df['cumulative_pnl']

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Cumulative PnL with running max
axes[0].plot(trades_df['timestamp'], trades_df['cumulative_pnl'], label='Cumulative PnL', linewidth=2)
axes[0].plot(trades_df['timestamp'], trades_df['running_max'], label='Running Maximum', 
             linestyle='--', linewidth=2, alpha=0.7)
axes[0].fill_between(trades_df['timestamp'], trades_df['cumulative_pnl'], 
                     trades_df['running_max'], alpha=0.3, color='red', label='Drawdown')
axes[0].set_title('Cumulative PnL and Drawdown', fontsize=14, fontweight='bold')
axes[0].set_ylabel('PnL (USD)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown over time
axes[1].fill_between(trades_df['timestamp'], 0, trades_df['drawdown'], alpha=0.5, color='red')
axes[1].set_title('Drawdown Over Time', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Drawdown (USD)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nMax Drawdown: ${metrics['max_drawdown']:.2f}")
print(f"Max Drawdown %: {metrics['max_drawdown_pct']:.2f}%")

## 8. Summary Statistics

In [ ]:
# Create summary DataFrame
summary = pd.DataFrame({
    'Metric': [
        'Starting Balance',
        'Final Balance',
        'Total PnL',
        'Return %',
        'Total Trades',
        'Winning Trades',
        'Losing Trades',
        'Win Rate %',
        'Avg Win',
        'Avg Loss',
        'Max Drawdown',
        'Sharpe Ratio',
        'Accuracy %'
    ],
    'Value': [
        f"${10000.0:.2f}",
        f"${metrics['final_balance']:.2f}",
        f"${metrics['total_pnl']:.2f}",
        f"{metrics['return_pct']:.2f}%",
        metrics['num_trades'],
        metrics['num_wins'],
        metrics['num_losses'],
        f"{metrics['win_rate']:.2f}%",
        f"${metrics['avg_win']:.2f}",
        f"${metrics['avg_loss']:.2f}",
        f"${metrics['max_drawdown']:.2f}",
        f"{metrics['sharpe_ratio']:.2f}",
        f"{metrics['accuracy']:.2f}%"
    ]
})

display(summary)

## 9. Export Results

In [ ]:
# Save trades to CSV
output_file = f'../results_analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
trades_df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")